In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from skimage import feature
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Function to load images and labels from a dataset folder
def load_dataset(dataset_path):
    images = []
    labels = []
    label_names = os.listdir(dataset_path)
    for label in label_names:
        label_path = os.path.join(dataset_path, label)
        for image_name in os.listdir(label_path):
            image_path = os.path.join(label_path, image_name)
            image = cv2.imread(image_path)
            images.append(image)
            labels.append(label)
    return images, labels

# Load dataset
dataset_path = 'fish_dt'  # Replace with your actual dataset path
images, labels = load_dataset(dataset_path)

In [3]:
def augment_images(images, labels, augment_size=1000):
    augmented_images = []
    augmented_labels = []
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    for image, label in zip(images, labels):
        image = np.expand_dims(image, axis=0)
        label = np.expand_dims(label, axis=0)
        aug_iter = datagen.flow(image, batch_size=1)
        for i in range(augment_size):
            augmented_image = aug_iter.next()[0].astype(np.uint8)
            augmented_images.append(augmented_image)
            augmented_labels.append(label)
    return np.array(augmented_images), np.array(augmented_labels)

In [4]:
# Convert images to grayscale
gray_images = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in images]

In [5]:
# Apply LBP
def apply_lbp(image, num_points=24, radius=8):
    lbp = local_binary_pattern(image, num_points, radius, method='uniform')
    return lbp

# Apply LDP
def apply_ldp(image):
    gx = cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize=3)
    magnitude, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
    angle_bins = np.int32(8 * angle / 360)  # Quantize the angle to 8 bins
    return angle_bins

In [6]:
# Compute histogram
def compute_histogram(image, bins=16):
    hist, _ = np.histogram(image.ravel(), bins=bins, range=(0, bins))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)  # Normalize the histogram
    return hist

In [7]:
# Apply BWT (Block-wise Truncation)
def apply_bwt(image, blocks=(8, 8)):
    features = []
    h, w = image.shape
    (bx, by) = blocks
    xsteps = np.linspace(0, w, bx + 1, dtype=int)
    ysteps = np.linspace(0, h, by + 1, dtype=int)
    for i in range(1, len(ysteps)):
        for j in range(1, len(xsteps)):
            block = image[ysteps[i-1]:ysteps[i], xsteps[j-1]:xsteps[j]]
            hist = np.histogram(block, bins=256, range=(0, 256))[0] / (block.shape[0] * block.shape[1])
            features.extend(hist)
    return features

# Apply Hough Transform
def apply_hough(image):
    edges = cv2.Canny(image, 50, 150, apertureSize=3)
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)
    return len(lines) if lines is not None else 0

In [8]:
# Extract features from the dataset
lbp_features_list = []
ldp_features_list = []
bwt_features_list = []
hough_features_list = []

for gray_image in gray_images:
    # Apply LBP
    lbp_image = apply_lbp(gray_image)
    lbp_hist_features = compute_histogram(lbp_image, bins=16)
    lbp_features_list.append(lbp_hist_features)
    
    # Apply LDP
    ldp_image = apply_ldp(gray_image)
    ldp_hist_features = compute_histogram(ldp_image, bins=16)
    ldp_features_list.append(ldp_hist_features)
    
    # Apply BWT
    bwt_features = apply_bwt(gray_image)
    bwt_features_list.append(bwt_features)
    
    # Apply Hough Transform
    hough_features = apply_hough(gray_image)
    hough_features_list.append(hough_features)

In [9]:
# Convert labels to numerical values
label_encoder = LabelEncoder()
numeric_labels = label_encoder.fit_transform(labels)

In [10]:
# Function to train and evaluate classifiers
def train_and_evaluate_model(X_train, X_test, y_train, y_test, classifier):
    clf = classifier
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"Classifier: {clf.__class__.__name__}")
    print(f"Classification Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

In [11]:
# Split the data into training and testing sets for LBP features
X_train, X_test, y_train, y_test = train_test_split(lbp_features_list, numeric_labels, test_size=0.2, random_state=42)
print("Evaluating LBP Features")
for clf in [SVC(kernel='linear'), RandomForestClassifier(n_estimators=100, random_state=42),
            GradientBoostingClassifier(n_estimators=100, random_state=42), AdaBoostClassifier(n_estimators=100, random_state=42),
            DecisionTreeClassifier(random_state=42), LogisticRegression(), GaussianNB()]:
    train_and_evaluate_model(X_train, X_test, y_train, y_test, clf)

Evaluating LBP Features
Classifier: SVC
Classification Accuracy: 0.8714285714285714
Precision: 0.7593877551020408
Recall: 0.8714285714285714
F1 Score: 0.8115594329334788
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        45
           1       0.87      1.00      0.93       305

    accuracy                           0.87       350
   macro avg       0.44      0.50      0.47       350
weighted avg       0.76      0.87      0.81       350



C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

Classifier: RandomForestClassifier
Classification Accuracy: 0.8885714285714286
Precision: 0.8704978354978355
Recall: 0.8885714285714286
F1 Score: 0.8693363329583802
Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.29      0.40        45
           1       0.90      0.98      0.94       305

    accuracy                           0.89       350
   macro avg       0.78      0.63      0.67       350
weighted avg       0.87      0.89      0.87       350

Classifier: GradientBoostingClassifier
Classification Accuracy: 0.8942857142857142
Precision: 0.8817579127459367
Recall: 0.8942857142857142
F1 Score: 0.8715557461343656
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.27      0.39        45
           1       0.90      0.99      0.94       305

    accuracy                           0.89       350
   macro avg       0.83      0.63      0.67       350
weighted avg       0.

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Classifier: AdaBoostClassifier
Classification Accuracy: 0.8571428571428571
Precision: 0.8435374149659864
Recall: 0.8571428571428571
F1 Score: 0.8493663594470046
Classification Report:
              precision    recall  f1-score   support

           0       0.43      0.33      0.38        45
           1       0.90      0.93      0.92       305

    accuracy                           0.86       350
   macro avg       0.67      0.63      0.65       350
weighted avg       0.84      0.86      0.85       350

Classifier: DecisionTreeClassifier
Classification Accuracy: 0.8085714285714286
Precision: 0.8241473616473616
Recall: 0.8085714285714286
F1 Score: 0.8158394018626964
Classification Report:
              precision    recall  f1-score   support

           0       0.30      0.36      0.32        45
           1       0.90      0.88      0.89       305

    accuracy                           0.81       350
   macro avg       0.60      0.62      0.61       350
weighted avg       0.82      

In [12]:
# Split the data into training and testing sets for LDP features
X_train, X_test, y_train, y_test = train_test_split(ldp_features_list, numeric_labels, test_size=0.2, random_state=42)
print("\nEvaluating LDP Features")
for clf in [SVC(kernel='linear'), RandomForestClassifier(n_estimators=100, random_state=42),
            GradientBoostingClassifier(n_estimators=100, random_state=42), AdaBoostClassifier(n_estimators=100, random_state=42),
            DecisionTreeClassifier(random_state=42), LogisticRegression(), GaussianNB()]:
    train_and_evaluate_model(X_train, X_test, y_train, y_test, clf)


Evaluating LDP Features
Classifier: SVC
Classification Accuracy: 0.8714285714285714
Precision: 0.7593877551020408
Recall: 0.8714285714285714
F1 Score: 0.8115594329334788
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        45
           1       0.87      1.00      0.93       305

    accuracy                           0.87       350
   macro avg       0.44      0.50      0.47       350
weighted avg       0.76      0.87      0.81       350



C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

Classifier: RandomForestClassifier
Classification Accuracy: 0.8628571428571429
Precision: 0.806954335986594
Recall: 0.8628571428571429
F1 Score: 0.8209641751437418
Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.07      0.11        45
           1       0.88      0.98      0.93       305

    accuracy                           0.86       350
   macro avg       0.61      0.52      0.52       350
weighted avg       0.81      0.86      0.82       350

Classifier: GradientBoostingClassifier
Classification Accuracy: 0.8657142857142858
Precision: 0.812625313283208
Recall: 0.8657142857142858
F1 Score: 0.8226806700633652
Classification Report:
              precision    recall  f1-score   support

           0       0.38      0.07      0.11        45
           1       0.88      0.98      0.93       305

    accuracy                           0.87       350
   macro avg       0.63      0.53      0.52       350
weighted avg       0.81

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Classifier: AdaBoostClassifier
Classification Accuracy: 0.8485714285714285
Precision: 0.7715126050420169
Recall: 0.8485714285714285
F1 Score: 0.8044981375213933
Classification Report:
              precision    recall  f1-score   support

           0       0.10      0.02      0.04        45
           1       0.87      0.97      0.92       305

    accuracy                           0.85       350
   macro avg       0.49      0.50      0.48       350
weighted avg       0.77      0.85      0.80       350

Classifier: DecisionTreeClassifier
Classification Accuracy: 0.7714285714285715
Precision: 0.7914153642967202
Recall: 0.7714285714285715
F1 Score: 0.780952380952381
Classification Report:
              precision    recall  f1-score   support

           0       0.18      0.22      0.20        45
           1       0.88      0.85      0.87       305

    accuracy                           0.77       350
   macro avg       0.53      0.54      0.53       350
weighted avg       0.79      0

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

In [13]:
# Combine LBP and LDP features
combined_features_list = [np.hstack((lbp, ldp)) for lbp, ldp in zip(lbp_features_list, ldp_features_list)]

# Split the data into training and testing sets for combined features
X_train, X_test, y_train, y_test = train_test_split(combined_features_list, numeric_labels, test_size=0.2, random_state=42)
print("\nEvaluating Combined LBP and LDP Features")
for clf in [SVC(kernel='linear'), RandomForestClassifier(n_estimators=100, random_state=42),
            GradientBoostingClassifier(n_estimators=100, random_state=42), AdaBoostClassifier(n_estimators=100, random_state=42),
            DecisionTreeClassifier(random_state=42), LogisticRegression(), GaussianNB()]:
    train_and_evaluate_model(X_train, X_test, y_train, y_test, clf)



Evaluating Combined LBP and LDP Features
Classifier: SVC
Classification Accuracy: 0.8714285714285714
Precision: 0.7593877551020408
Recall: 0.8714285714285714
F1 Score: 0.8115594329334788
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        45
           1       0.87      1.00      0.93       305

    accuracy                           0.87       350
   macro avg       0.44      0.50      0.47       350
weighted avg       0.76      0.87      0.81       350



C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

Classifier: RandomForestClassifier
Classification Accuracy: 0.8942857142857142
Precision: 0.8817579127459367
Recall: 0.8942857142857142
F1 Score: 0.8715557461343656
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.27      0.39        45
           1       0.90      0.99      0.94       305

    accuracy                           0.89       350
   macro avg       0.83      0.63      0.67       350
weighted avg       0.88      0.89      0.87       350

Classifier: GradientBoostingClassifier
Classification Accuracy: 0.8914285714285715
Precision: 0.8750325662179765
Recall: 0.8914285714285715
F1 Score: 0.8737432914089066
Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.31      0.42        45
           1       0.91      0.98      0.94       305

    accuracy                           0.89       350
   macro avg       0.79      0.64      0.68       350
weighted avg       0.

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Classifier: AdaBoostClassifier
Classification Accuracy: 0.88
Precision: 0.8713237198860202
Recall: 0.88
F1 Score: 0.8749230404925408
Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.44      0.49        45
           1       0.92      0.94      0.93       305

    accuracy                           0.88       350
   macro avg       0.73      0.69      0.71       350
weighted avg       0.87      0.88      0.87       350

Classifier: DecisionTreeClassifier
Classification Accuracy: 0.8142857142857143
Precision: 0.8327259475218659
Recall: 0.8142857142857143
F1 Score: 0.8226936454675756
Classification Report:
              precision    recall  f1-score   support

           0       0.32      0.40      0.36        45
           1       0.91      0.88      0.89       305

    accuracy                           0.81       350
   macro avg       0.61      0.64      0.62       350
weighted avg       0.83      0.81      0.82       350

Cl

In [14]:
# Split the data into training and testing sets for BWT features
X_train, X_test, y_train, y_test = train_test_split(bwt_features_list, numeric_labels, test_size=0.2, random_state=42)
print("\nEvaluating BWT Features")
for clf in [SVC(kernel='linear'), RandomForestClassifier(n_estimators=100, random_state=42),
            GradientBoostingClassifier(n_estimators=100, random_state=42), AdaBoostClassifier(n_estimators=100, random_state=42),
            DecisionTreeClassifier(random_state=42), LogisticRegression(), GaussianNB()]:
    train_and_evaluate_model(X_train, X_test, y_train, y_test, clf)


Evaluating BWT Features
Classifier: SVC
Classification Accuracy: 0.8828571428571429
Precision: 0.8635626643295355
Recall: 0.8828571428571429
F1 Score: 0.8669803427468967
Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.31      0.41        45
           1       0.90      0.97      0.94       305

    accuracy                           0.88       350
   macro avg       0.74      0.64      0.67       350
weighted avg       0.86      0.88      0.87       350

Classifier: RandomForestClassifier
Classification Accuracy: 0.8885714285714286
Precision: 0.901204318936877
Recall: 0.8885714285714286
F1 Score: 0.8493143944789009
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.13      0.24        45
           1       0.89      1.00      0.94       305

    accuracy                           0.89       350
   macro avg       0.94      0.57      0.59       350
weighted avg       0

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Classifier: AdaBoostClassifier
Classification Accuracy: 0.8942857142857142
Precision: 0.8810115350488021
Recall: 0.8942857142857142
F1 Score: 0.8834096449092843
Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.40      0.49        45
           1       0.92      0.97      0.94       305

    accuracy                           0.89       350
   macro avg       0.78      0.68      0.72       350
weighted avg       0.88      0.89      0.88       350

Classifier: DecisionTreeClassifier
Classification Accuracy: 0.8171428571428572
Precision: 0.8205703853060079
Recall: 0.8171428571428572
F1 Score: 0.8188296829029095
Classification Report:
              precision    recall  f1-score   support

           0       0.30      0.31      0.30        45
           1       0.90      0.89      0.89       305

    accuracy                           0.82       350
   macro avg       0.60      0.60      0.60       350
weighted avg       0.82      

In [15]:
print("Shape of X_train:", np.array(X_train).shape)
print("Shape of y_train:", np.array(y_train).shape)


Shape of X_train: (1400, 16384)
Shape of y_train: (1400,)


In [16]:
hough_features_list = np.array(hough_features_list).reshape(-1, 1)


In [17]:
# Split the data into training and testing sets for combined features
combined_features_list = [np.hstack((lbp, ldp, bwt, hough)) for lbp, ldp, bwt, hough in
                          zip(lbp_features_list, ldp_features_list, bwt_features_list, hough_features_list)]
X_train, X_test, y_train, y_test = train_test_split(combined_features_list, numeric_labels, test_size=0.2, random_state=42)
print("\nEvaluating Combined LBP, LDP, BWT, and Hough Features")
for clf in [SVC(kernel='linear'), RandomForestClassifier(n_estimators=100, random_state=42),
            GradientBoostingClassifier(n_estimators=100, random_state=42), AdaBoostClassifier(n_estimators=100, random_state=42),
            DecisionTreeClassifier(random_state=42), LogisticRegression(), GaussianNB()]:
    train_and_evaluate_model(X_train, X_test, y_train, y_test, clf)



Evaluating Combined LBP, LDP, BWT, and Hough Features
Classifier: SVC
Classification Accuracy: 0.8914285714285715
Precision: 0.8792048885486038
Recall: 0.8914285714285715
F1 Score: 0.8826465201465201
Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.42      0.50        45
           1       0.92      0.96      0.94       305

    accuracy                           0.89       350
   macro avg       0.77      0.69      0.72       350
weighted avg       0.88      0.89      0.88       350

Classifier: RandomForestClassifier
Classification Accuracy: 0.8942857142857142
Precision: 0.9057226399331663
Recall: 0.8942857142857142
F1 Score: 0.8604081870711598
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.18      0.30        45
           1       0.89      1.00      0.94       305

    accuracy                           0.89       350
   macro avg       0.95      0.59      0.62

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Classifier: AdaBoostClassifier
Classification Accuracy: 0.8914285714285715
Precision: 0.8806259986070711
Recall: 0.8914285714285715
F1 Score: 0.8841242358927247
Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.44      0.51        45
           1       0.92      0.96      0.94       305

    accuracy                           0.89       350
   macro avg       0.76      0.70      0.73       350
weighted avg       0.88      0.89      0.88       350

Classifier: DecisionTreeClassifier
Classification Accuracy: 0.8285714285714286
Precision: 0.8349718624991523
Recall: 0.8285714285714286
F1 Score: 0.8316530741220018
Classification Report:
              precision    recall  f1-score   support

           0       0.35      0.38      0.36        45
           1       0.91      0.90      0.90       305

    accuracy                           0.83       350
   macro avg       0.63      0.64      0.63       350
weighted avg       0.83      

In [18]:
# Function to extract features from RGB channels
def extract_color_features(image):
    channels = cv2.split(image)
    features = []
    for channel in channels:
        lbp_image = apply_lbp(channel)
        lbp_hist_features = compute_histogram(lbp_image, bins=16)
        ldp_image = apply_ldp(channel)
        ldp_hist_features = compute_histogram(ldp_image, bins=16)
        features.extend(np.hstack((lbp_hist_features, ldp_hist_features)))
    return features

# Extract features from RGB channels
rgb_features_list = [extract_color_features(image) for image in images]

In [19]:
# Split the data into training and testing sets for RGB features
X_train, X_test, y_train, y_test = train_test_split(rgb_features_list, numeric_labels, test_size=0.2, random_state=42)
print("\nEvaluating RGB Channel Features")
for clf in [SVC(kernel='linear'), RandomForestClassifier(n_estimators=100, random_state=42),
            GradientBoostingClassifier(n_estimators=100, random_state=42), AdaBoostClassifier(n_estimators=100, random_state=42),
            DecisionTreeClassifier(random_state=42), LogisticRegression(), GaussianNB()]:
    train_and_evaluate_model(X_train, X_test, y_train, y_test, clf)



Evaluating RGB Channel Features
Classifier: SVC
Classification Accuracy: 0.8714285714285714
Precision: 0.7593877551020408
Recall: 0.8714285714285714
F1 Score: 0.8115594329334788
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        45
           1       0.87      1.00      0.93       305

    accuracy                           0.87       350
   macro avg       0.44      0.50      0.47       350
weighted avg       0.76      0.87      0.81       350



C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

Classifier: RandomForestClassifier
Classification Accuracy: 0.8971428571428571
Precision: 0.8838037342596614
Recall: 0.8971428571428571
F1 Score: 0.8803883813347535
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.33      0.45        45
           1       0.91      0.98      0.94       305

    accuracy                           0.90       350
   macro avg       0.81      0.66      0.70       350
weighted avg       0.88      0.90      0.88       350

Classifier: GradientBoostingClassifier
Classification Accuracy: 0.88
Precision: 0.856352361265702
Recall: 0.88
F1 Score: 0.8580778301886792
Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.24      0.34        45
           1       0.90      0.97      0.93       305

    accuracy                           0.88       350
   macro avg       0.74      0.61      0.64       350
weighted avg       0.86      0.88      0.86       

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Classifier: AdaBoostClassifier
Classification Accuracy: 0.8542857142857143
Precision: 0.8445657412762677
Recall: 0.8542857142857143
F1 Score: 0.8489677161993656
Classification Report:
              precision    recall  f1-score   support

           0       0.42      0.36      0.39        45
           1       0.91      0.93      0.92       305

    accuracy                           0.85       350
   macro avg       0.66      0.64      0.65       350
weighted avg       0.84      0.85      0.85       350

Classifier: DecisionTreeClassifier
Classification Accuracy: 0.7885714285714286
Precision: 0.8278285503326377
Recall: 0.7885714285714286
F1 Score: 0.8053814382896015
Classification Report:
              precision    recall  f1-score   support

           0       0.28      0.42      0.34        45
           1       0.91      0.84      0.87       305

    accuracy                           0.79       350
   macro avg       0.60      0.63      0.61       350
weighted avg       0.83      

In [20]:
# Function to convert to different color spaces and extract features
def extract_color_space_features(image, color_space):
    if color_space == 'HSV':
        converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    elif color_space == 'LAB':
        converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    elif color_space == 'YCbCr':
        converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    else:
        raise ValueError("Unsupported color space")
    return extract_color_features(converted_image)

# Extract features from different color spaces
color_spaces = ['HSV', 'LAB', 'YCbCr']
for color_space in color_spaces:
    color_space_features_list = [extract_color_space_features(image, color_space) for image in images]
    X_train, X_test, y_train, y_test = train_test_split(color_space_features_list, numeric_labels, test_size=0.2, random_state=42)
    print(f"\nEvaluating {color_space} Color Space Features")
    for clf in [SVC(kernel='linear'), RandomForestClassifier(n_estimators=100, random_state=42),
                GradientBoostingClassifier(n_estimators=100, random_state=42), AdaBoostClassifier(n_estimators=100, random_state=42),
                DecisionTreeClassifier(random_state=42), LogisticRegression(), GaussianNB()]:
        train_and_evaluate_model(X_train, X_test, y_train, y_test, clf)



Evaluating HSV Color Space Features
Classifier: SVC
Classification Accuracy: 0.8714285714285714
Precision: 0.7593877551020408
Recall: 0.8714285714285714
F1 Score: 0.8115594329334788
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        45
           1       0.87      1.00      0.93       305

    accuracy                           0.87       350
   macro avg       0.44      0.50      0.47       350
weighted avg       0.76      0.87      0.81       350



C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

Classifier: RandomForestClassifier
Classification Accuracy: 0.8942857142857142
Precision: 0.8802925989672977
Recall: 0.8942857142857142
F1 Score: 0.8738730657098003
Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.29      0.41        45
           1       0.90      0.98      0.94       305

    accuracy                           0.89       350
   macro avg       0.81      0.64      0.68       350
weighted avg       0.88      0.89      0.87       350

Classifier: GradientBoostingClassifier
Classification Accuracy: 0.9028571428571428
Precision: 0.8916043956043956
Recall: 0.9028571428571428
F1 Score: 0.8905215419501132
Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.40      0.51        45
           1       0.92      0.98      0.95       305

    accuracy                           0.90       350
   macro avg       0.82      0.69      0.73       350
weighted avg       0.

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Classifier: AdaBoostClassifier
Classification Accuracy: 0.8685714285714285
Precision: 0.863642411736184
Recall: 0.8685714285714285
F1 Score: 0.8659430562619715
Classification Report:
              precision    recall  f1-score   support

           0       0.49      0.44      0.47        45
           1       0.92      0.93      0.93       305

    accuracy                           0.87       350
   macro avg       0.70      0.69      0.70       350
weighted avg       0.86      0.87      0.87       350

Classifier: DecisionTreeClassifier
Classification Accuracy: 0.82
Precision: 0.8283333333333333
Recall: 0.82
F1 Score: 0.8239930404523707
Classification Report:
              precision    recall  f1-score   support

           0       0.32      0.36      0.34        45
           1       0.90      0.89      0.90       305

    accuracy                           0.82       350
   macro avg       0.61      0.62      0.62       350
weighted avg       0.83      0.82      0.82       350

Cla

...

Classifier: AdaBoostClassifier
Classification Accuracy: 0.8942857142857142
Precision: 0.8846558876715244
Recall: 0.8942857142857142
F1 Score: 0.8878621368439749
Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.47      0.53        45
           1       0.92      0.96      0.94       305

    accuracy                           0.89       350
   macro avg       0.77      0.71      0.74       350
weighted avg       0.88      0.89      0.89       350

Classifier: DecisionTreeClassifier
Classification Accuracy: 0.8228571428571428
Precision: 0.8390402817521462
Recall: 0.8228571428571428
F1 Score: 0.8302380952380952
Classification Report:
              precision    recall  f1-score   support

           0       0.35      0.42      0.38        45
           1       0.91      0.88      0.90       305

    accuracy                           0.82       350
   macro avg       0.63      0.65      0.64       350
weighted avg       0.84      

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

Classifier: RandomForestClassifier
Classification Accuracy: 0.9
Precision: 0.887888026607539
Recall: 0.9
F1 Score: 0.8846525665511307
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.36      0.48        45
           1       0.91      0.98      0.94       305

    accuracy                           0.90       350
   macro avg       0.82      0.67      0.71       350
weighted avg       0.89      0.90      0.88       350

Classifier: GradientBoostingClassifier
Classification Accuracy: 0.9085714285714286
Precision: 0.8994285714285715
Recall: 0.9085714285714286
F1 Score: 0.8969614512471655
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.42      0.54        45
           1       0.92      0.98      0.95       305

    accuracy                           0.91       350
   macro avg       0.84      0.70      0.75       350
weighted avg       0.90      0.91      0.90       35

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Classifier: AdaBoostClassifier
Classification Accuracy: 0.9142857142857143
Precision: 0.907936507936508
Recall: 0.9142857142857143
F1 Score: 0.9096198156682028
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.56      0.62        45
           1       0.94      0.97      0.95       305

    accuracy                           0.91       350
   macro avg       0.83      0.76      0.79       350
weighted avg       0.91      0.91      0.91       350

Classifier: DecisionTreeClassifier
Classification Accuracy: 0.8371428571428572
Precision: 0.8507722007722007
Recall: 0.8371428571428572
F1 Score: 0.8433260583010999
Classification Report:
              precision    recall  f1-score   support

           0       0.39      0.47      0.42        45
           1       0.92      0.89      0.91       305

    accuracy                           0.84       350
   macro avg       0.65      0.68      0.66       350
weighted avg       0.85      0